In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Constants
DELTA_T = 0.1  # Time step (seconds)
AREA_LIMIT = 3.0  # 3x3 square
NUM_PARTICLES = 500  # Number of particles
MOTION_NOISE = [0.1, 0.1, np.deg2rad(1)]  # Noise in [x, y, theta]
LANDMARKS = [[0.5, 0.5], [2.5, 0.5], [2.5, 2.5], [0.5, 2.5]]  # Landmarks in the area

# Load tf_data (ground truth as input for now)
def load_tf_data(filename):
    """Load tf_data (ground truth) from a CSV file."""
    df = pd.read_csv(filename)
    return df['translation_x'], df['translation_y'], df['rotation_z']

# Particle Filter Class
class ParticleFilter:
    def __init__(self, num_particles, area_limit):
        self.num_particles = num_particles
        self.area_limit = area_limit
        # Initialize particles uniformly within the area
        self.particles = np.zeros((num_particles, 3))  # [x, y, theta]
        self.particles[:, 0] = np.random.uniform(0, area_limit, num_particles)  # x
        self.particles[:, 1] = np.random.uniform(0, area_limit, num_particles)  # y
        self.particles[:, 2] = np.random.uniform(-np.pi, np.pi, num_particles)  # theta
        self.weights = np.ones(num_particles) / num_particles

    def predict(self, control_input):
        """Propagate particles using the motion model with noise."""
        v, omega = control_input  # Linear and angular velocity
        noise = np.random.normal(0, MOTION_NOISE, (self.num_particles, 3))
        self.particles[:, 2] += omega * DELTA_T + noise[:, 2]  # Update theta
        self.particles[:, 2] = (self.particles[:, 2] + np.pi) % (2 * np.pi) - np.pi  # Normalize angle
        self.particles[:, 0] += (v * np.cos(self.particles[:, 2]) * DELTA_T + noise[:, 0])  # Update x
        self.particles[:, 1] += (v * np.sin(self.particles[:, 2]) * DELTA_T + noise[:, 1])  # Update y
        # Clamp particles to the area limit
        self.particles[:, 0] = np.clip(self.particles[:, 0], 0, self.area_limit)
        self.particles[:, 1] = np.clip(self.particles[:, 1], 0, self.area_limit)

    def update(self, measurements, measurement_noise, landmarks):
        """Update particle weights based on measurements to landmarks."""
        for i, landmark in enumerate(landmarks):
            # Calculate expected distances to this landmark for all particles
            expected_distances = np.sqrt((self.particles[:, 0] - landmark[0])**2 + 
                                         (self.particles[:, 1] - landmark[1])**2)
            # Actual distance measurement for this landmark
            actual_distance = measurements[i]
            
            # Calculate weights based on Gaussian likelihood
            self.weights *= np.exp(-0.5 * ((expected_distances - actual_distance) / measurement_noise[0])**2)
        
        # Normalize weights
        self.weights /= np.sum(self.weights)

    def resample(self):
        """Resample particles based on their weights."""
        indices = np.random.choice(range(self.num_particles), size=self.num_particles, p=self.weights)
        self.particles = self.particles[indices]
        self.weights.fill(1.0 / self.num_particles)  # Reset weights

    def estimate(self):
        """Estimate the state as the weighted mean of the particles."""
        mean = np.average(self.particles, weights=self.weights, axis=0)
        covariance = np.cov(self.particles.T, aweights=self.weights)
        return mean, covariance

# Visualization Function
def plot_particles(particles, weights, ground_truth):
    """Plot particles, their weights, and the ground truth position."""
    plt.figure(figsize=(8, 8))
    plt.scatter(particles[:, 0], particles[:, 1], s=weights * 1000, alpha=0.4, label='Particles')
    plt.scatter(ground_truth[0], ground_truth[1], color='red', label='Ground Truth', s=100)
    for landmark in LANDMARKS:
        plt.scatter(landmark[0], landmark[1], color='green', label='Landmark', s=100)
    plt.xlim(-0.5, AREA_LIMIT + 0.5)
    plt.ylim(-0.5, AREA_LIMIT + 0.5)
    plt.xlabel('X Position (m)')
    plt.ylabel('Y Position (m)')
    plt.legend(loc='upper right')
    plt.grid()
    plt.show()

# Simulate Landmark Measurements
def simulate_landmark_measurements(ground_truth, landmarks, noise_std):
    """Simulate measurements from the ground truth position to landmarks."""
    measurements = []
    for landmark in landmarks:
        distance = np.sqrt((ground_truth[0] - landmark[0])**2 + (ground_truth[1] - landmark[1])**2)
        noisy_distance = distance + np.random.normal(0, noise_std)
        measurements.append(noisy_distance)
    return measurements

# Main Simulation Example
# Load tf_data (assuming the file is named 'tf_data.csv')
tf_x, tf_y, tf_theta = load_tf_data('tf_data.csv')

# Initialize Particle Filter
pf = ParticleFilter(NUM_PARTICLES, AREA_LIMIT)

# Simulate Particle Filter with tf_data
ground_truth = np.vstack((tf_x, tf_y, tf_theta)).T
for t in range(len(ground_truth) - 1):
    # Use ground truth motion as control input (for now)
    control_input = [0.5, np.deg2rad(5)]  # Example control input (v, omega)

    # Prediction Step
    pf.predict(control_input)

    # Simulate landmark measurements from the ground truth
    measurement = simulate_landmark_measurements(ground_truth[t], LANDMARKS, noise_std=0.2)

    # Update Step
    pf.update(measurement, measurement_noise=[0.2], landmarks=LANDMARKS)

    # Resampling Step
    pf.resample()

    # Visualization
    plot_particles(pf.particles, pf.weights, ground_truth[t, :2])

# Final Estimate
final_mean, final_covariance = pf.estimate()
print("Final Estimated State:", final_mean)
print("Final Estimated Covariance:", final_covariance)
